In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
import datetime as dt
from calendar import monthrange

def nearest_end_of_month(timestamp: dt.datetime) -> dt.datetime:
    if timestamp.day < 15:
        timestamp = timestamp - dt.timedelta(days=15)
    month_end = monthrange(timestamp.year, timestamp.month)[1]
    return timestamp.replace(day=month_end)

In [25]:
pd.set_option('display.max_rows', 2048)

In [4]:
prices = pd.read_csv('data/weekly_prices.csv')
prices

,tick,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount
0,AAPL,2023-12-18,196.09,196.63,194.3900,195.89,195.8900,54182148,0.0
1,AAPL,2023-12-15,193.11,199.62,191.4200,197.57,197.5700,379414755,0.0
2,AAPL,2023-12-08,189.98,195.99,187.4511,195.71,195.7100,251797045,0.0
3,AAPL,2023-12-01,189.92,192.09,188.1900,191.24,191.2400,216481441,0.0
4,AAPL,2023-11-24,189.89,192.93,189.2500,189.97,189.9700,148351454,0.0
...,...,...,...,...,...,...,...,...,...
10066,TSM,1999-12-10,39.12,40.06,37.5600,38.50,14.3000,8060100,0.0
10067,TSM,1999-12-03,36.56,41.00,35.8100,40.69,15.1135,10031000,0.0
10068,TSM,1999-11-26,37.12,37.44,35.8800,36.37,13.5089,6241700,0.0
10069,TSM,1999-11-19,37.00,38.00,36.6200,37.12,13.7875,8907100,0.0


In [10]:
eps = pd.read_csv('data/quarterly_eps.csv')
eps['timestamp'] = pd.to_datetime(eps.timestamp)
eps['reported_Date'] = pd.to_datetime(eps.reported_Date)
for col in eps.columns[3:]:
    eps[col] = pd.to_numeric(eps[col], 'coerce')
#pd.to_datetime(eps.timestamp).dt.day.value_counts()
eps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tick                 831 non-null    object        
 1   timestamp            831 non-null    datetime64[ns]
 2   reported_Date        831 non-null    datetime64[ns]
 3   reported_EPS         831 non-null    float64       
 4   estimated_EPS        815 non-null    float64       
 5   surprise             815 non-null    float64       
 6   surprise_Percentage  814 non-null    float64       
dtypes: datetime64[ns](2), float64(4), object(1)
memory usage: 45.6+ KB


In [23]:
income = pd.read_csv('data/quarterly_income.csv')
income['timestamp'] = pd.to_datetime(income.timestamp)
income['timestamp'] = income.timestamp.apply(nearest_end_of_month)
for col in income.columns[2:]:
    income[col] = pd.to_numeric(income[col], 'coerce')
income

,tick,timestamp,net_Income,total_Revenue,total_Expenses,profit_Margin
0,AAPL,2023-09-30,22956000000,88496000000,65540000000,0.259402
1,AAPL,2023-06-30,19881000000,80799000000,60918000000,0.246055
2,AAPL,2023-03-31,24160000000,94836000000,70676000000,0.254756
3,AAPL,2022-12-31,29998000000,116151000000,86153000000,0.258267
4,AAPL,2022-09-30,20721000000,89319000000,68598000000,0.231989
...,...,...,...,...,...,...
177,TSM,2019-09-30,101069886000,293025712000,191955826000,0.344918
178,TSM,2019-06-30,66764850000,241055305000,174290455000,0.276969
179,TSM,2019-03-31,99984000000,218709901000,118725901000,0.457154
180,TSM,2018-12-31,99984000000,289771000000,189787000000,0.345045


In [28]:
quarters = pd.merge(eps, income, 'right', on=['tick', 'timestamp']).set_index(['tick', 'timestamp'])
quarters

reported_Date  reported_EPS  estimated_EPS  surprise  \
tick timestamp                                                         
AAPL 2023-09-30    2023-11-02        1.4600         1.3900    0.0700   
     2023-06-30    2023-08-03        1.2600         1.1900    0.0700   
     2023-03-31    2023-05-04        1.5200         1.4300    0.0900   
     2022-12-31    2023-02-02        1.8800         1.9400   -0.0600   
     2022-09-30    2022-10-27        1.2900         1.2700    0.0200   
     2022-06-30    2022-07-28        1.2000         1.1600    0.0400   
     2022-03-31    2022-04-28        1.5200         1.4300    0.0900   
     2021-12-31    2022-01-27        2.1000         1.8900    0.2100   
     2021-09-30    2021-10-28        1.2400         1.2400    0.0000   
     2021-06-30    2021-07-27        1.3000         1.0100    0.2900   
     2021-03-31    2021-04-28        1.4000         0.9900    0.4100   
     2020-12-31    2021-01-27        1.6800         1.4100    0.2700   
     2020-09-30    2020-10-29        0.7300         0.7000    0.0300   
     2020-06-30    2020-07-30        0.6500         0.5100    0.1400   
     2020-03-31    2020-04-30        0.6400         0.5600    0.0800   
     2019-12-31    2020-01-28        1.2500         1.1400    0.1100   
     2019-09-30    2019-10-30        0.7600         0.7100    0.0500   
     2019-06-30    2019-07-30        0.5500         0.5300    0.0200   
     2019-03-31    2019-04-30        0.6200         0.5900    0.0300   
     2018-12-31    2019-01-29        1.0500         1.0400    0.0100   
IBM  2023-09-30    2023-10-25        2.2000         2.1300    0.0700   
     2023-06-30    2023-07-19        2.1800         2.0100    0.1700   
     2023-03-31    2023-04-19        1.3600         1.2600    0.1000   
     2022-12-31    2023-01-25        3.6000         3.6000    0.0000   
     2022-09-30    2022-10-19        1.8100         1.7700    0.0400   
     2022-06-30    2022-07-18        2.3100         2.2700    0.0400   
     2022-03-31    2022-04-19        1.4000         1.3800    0.0200   
     2021-12-31    2022-01-24        3.3500         3.3000    0.0500   
     2021-09-30    2021-10-20        2.5200         2.5000    0.0200   
     2021-06-30    2021-07-19        2.3300         2.2900    0.0400   
     2021-03-31    2021-04-19        1.7700         1.6300    0.1400   
     2020-12-31    2021-01-21        2.0700         1.7900    0.2800   
     2020-09-30    2020-10-19        2.5800         2.5800    0.0000   
     2020-06-30    2020-07-20        2.1800         2.0700    0.1100   
     2020-03-31    2020-04-20        1.8400         1.8000    0.0400   
     2019-12-31    2020-01-21        4.7100         4.6800    0.0300   
     2019-09-30    2019-10-16        2.6800         2.6700    0.0100   
     2019-06-30    2019-07-17        3.1700         3.0700    0.1000   
     2019-03-31    2019-04-16        2.2500         2.2200    0.0300   
     2018-12-31    2019-01-22        4.8700         4.8200    0.0500   
NVDA 2023-10-31    2023-11-21        4.0200         3.3700    0.6500   
     2023-07-31    2023-08-23        2.7000         2.0900    0.6100   
     2023-04-30    2023-05-24        1.0900         0.9200    0.1700   
     2023-01-31    2023-02-22        0.8800         0.8100    0.0700   
     2022-10-31    2022-11-16        0.5800         0.6900   -0.1100   
     2022-07-31    2022-08-24        0.5100         1.2600   -0.7500   
     2022-04-30    2022-05-25        1.3600         1.2900    0.0700   
     2022-01-31    2022-02-16        1.3200         1.2200    0.1000   
     2021-10-31    2021-11-17        1.1700         1.1100    0.0600   
     2021-07-31    2021-08-18        1.0400         1.0100    0.0300   
     2021-04-30    2021-05-26        0.9200         0.8200    0.1000   
     2021-01-31    2021-02-24        0.7800         0.7000    0.0800   
     2020-10-31    2020-11-18        0.7300         0.6400    0.0900   
     2020-07-31    2020-08-19        0.5500         0.49